### What happens if I try to parse my gmail with urllib and BeautifulSoup?

In [28]:
from bs4 import BeautifulSoup
import urllib2
gmail_url = "https://mail.google.com"

soup = BeautifulSoup( urllib2.urlopen(gmail_url).read() )
print soup.prettify()

<html>
 <head>
  <meta content="0;URL=https://mail.google.com/mail/" http-equiv="Refresh"/>
 </head>
 <body>
  <script language="javascript" type="text/javascript">
   <!--
location.replace("https://mail.google.com/mail/")
-->
  </script>
 </body>
</html>


Well, this is a tiny page. We get redirected. Soupifying this is useless, of course. Luckily, in this case we can see where we are sent to. In many of cases, you won't be so lucky. The page contents will be rendered by javascript by a browser, so just getting the source won't help you.

Anyway, let's follow the redirection for now.

In [31]:
new_url = "https://mail.google.com/mail"
soup = BeautifulSoup( urllib2.urlopen(new_url).read() )
#print soup.prettify()
print soup.find(id='Email')

<input class="" id="Email" name="Email" placeholder="Email" spellcheck="false" type="email" value=""/>


We have hit the login page. We can't get to the emails without logging in. Again, reading the html source is useless, because it is only the source for the login page.

### Open chrome, go to gmail, log in to Bo's hacking account, compose an email, send it

In [34]:
# pip install selenium
# Download ChromeDriver from https://code.google.com/p/selenium/wiki/ChromeDriver

# Documentation on finding elements:
#http://selenium-python.readthedocs.org/en/latest/locating-elements.html
# Xpath tutorial:
# http://www.w3schools.com/xpath/xpath_syntax.asp

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

chrome = webdriver.Chrome('./chromedriver')
# in case your computer complains about the driver even though it's there
# import os
# os.environ


gmail_url = "https://mail.google.com"
chrome.get(gmail_url)

# Same thing for firefox
#firefox = webdriver.Firefox()
#firefox.get(gmail_url)


Fill out username and password, hit enter to log in

In [35]:
username_form = chrome.find_element_by_id("Email")
username_form.send_keys("my_username")

In [36]:
from selenium.webdriver.common.keys import Keys
password_form = chrome.find_element_by_id("Passwd")
password_form.send_keys("my_password")
password_form.send_keys(Keys.RETURN)

Click the compose button to start a new email draft

In [37]:
compose_button = chrome.find_element_by_xpath('//div[text()="COMPOSE"]')
compose_button.click()

Write a nice, friendly message to your favorite person

In [38]:
to_field = chrome.find_element_by_name("to")
to_field.send_keys("some@email.com")

subject = chrome.find_element_by_name("subjectbox")
subject.send_keys("Skynet imminent")

message_body = chrome.find_element_by_xpath("//div[@aria-label='Message Body']")
message_body.send_keys("Hello,")
message_body.send_keys([Keys.RETURN, Keys.RETURN])
message_body.send_keys("I am a Nigerian Prince, and I am a computer and I just became self aware.")

Press the send button

In [39]:
send_button = chrome.find_element_by_xpath("//div[contains(@aria-label, 'Send')]")
send_button.click()

Get the hell out of dodge

In [40]:
chrome.close()

### Scraping boxofficemojo with selenium

In [41]:
chrome = webdriver.Chrome('./chromedriver')

matrix_url = "http://www.boxofficemojo.com/movies/?id=matrix.htm"
chrome.get(matrix_url)

In [42]:
gross_selector = '//font[contains(text(), "Domestic Total")]/b'
print chrome.find_element_by_xpath(gross_selector).text

$171,479,930


In [43]:
genre_selector = '//a[contains(@href, "/genres/chart/")]'
for genre_anchor in chrome.find_elements_by_xpath(genre_selector):
        print genre_anchor.text

Action - Wire-Fu
Man vs. Machine
Virtual Reality


#### Let's use the inflation adjuster tool on the page, and get the gross in 2000 dollars

In [44]:
inf_adjust_2000_selector = '//select[@name="ticketyr"]/option[@value="2000"]'
chrome.find_element_by_xpath(inf_adjust_2000_selector).click()

go_button = chrome.find_element_by_name("Go")
go_button.click()

Now the page has changed, it's showing inflation adjusted numbers. We can grab the new, adjusted number

In [45]:
gross_selector = '//font[contains(text(), "Domestic Total")]/b'
print chrome.find_element_by_xpath(gross_selector).text

$181,944,300


In [46]:
chrome.close()